In [93]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Run necessary background code and generate baseline

In [94]:
# Background functions and calculations of adjustment factors
execfile('import_packages.py')
execfile('read_in_data.py')
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')
execfile('usercode_taxcalc.py')

All packages successfully imported
Data imports complete
Beginning CCR model
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated


In [95]:
execfile('gen_baseline.py')

Taxable income calculated
Investment matrices constructed
Corporate depreciation calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
Noncorporate tax rates calculated
Baseline complete


# Specify tax reform

In [96]:
btax_dict1 = {
    2017: {
        'tau_c': 0.347,
        'tau_nc': 0.0,
        'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
iit_params_ref = {}
execfile('reform_implementation.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
All parameters updated


# Calculate the investment response

In [97]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': 0.0
}
execfile('responses.py')

In [98]:
len(run_btax_mini(range(2017, 2028), btax_defaults, brc_defaults_other))

96

In [99]:
execfile('mini_combined.py')
response_results = inv_response(2017)

In [100]:
len(get_btax_params_oneyear(btax_defaults, 2020))

96

In [101]:
response_results

,Asset,deltaIc2014,deltaInc2014,deltaEc2014,deltaEnc2014,deltaIc2015,deltaInc2015,deltaEc2015,deltaEnc2015,deltaIc2016,...,deltaEc2025,deltaEnc2025,deltaIc2026,deltaInc2026,deltaEc2026,deltaEnc2026,deltaIc2027,deltaInc2027,deltaEc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276570,0.273562,0.0,0.0,0.276570,0.273084,0.0,0.0,0.276570,0.274473
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.114346,0.101471,0.0,0.0,0.114346,0.101447,0.0,0.0,0.114346,0.101433
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.154959,0.148637,0.0,0.0,0.154959,0.148737,0.0,0.0,0.154959,0.148831
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.216570,0.213683,0.0,0.0,0.216570,0.213348,0.0,0.0,0.216570,0.214473
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.097106,0.091377,0.0,0.0,0.097106,0.091438,0.0,0.0,0.097106,0.091486
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.422672,0.410522,0.0,0.0,0.422672,0.410483,0.0,0.0,0.422672,0.410480
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.212754,0.199566,0.0,0.0,0.212754,0.199516,0.0,0.0,0.212754,0.199486
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.216570,0.213683,0.0,0.0,0.216570,0.213348,0.0,0.0,0.216570,0.214473
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099897,0.091309,0.0,0.0,0.099897,0.091316,0.0,0.0,0.099897,0.091321
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.189071,0.180652,0.0,0.0,0.189071,0.180705,0.0,0.0,0.189071,0.180756


# Run other responses and recalculate corporate tax revenue

In [102]:
execfile('gen_reform.py')

New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete


### Check capital results

In [104]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [105]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [106]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


In [107]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


### Check combined results

In [108]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2462.516902,884.575577,231.441998,34.306208,1312.193118,0.02574,0.347,421.555664,101.582376,4.457941,3.700228,320.731
1,2015,2537.690289,905.241147,243.981131,34.855388,1353.612624,0.02574,0.347,434.862110,91.794692,4.529305,3.799722,343.797
2,2016,2343.349034,830.188301,248.001094,34.816996,1230.342644,0.02574,0.347,395.260349,96.318143,4.524316,3.895522,299.571
3,2017,2444.411382,866.909916,268.261581,34.952204,1274.287681,0.02574,0.347,409.378148,99.935946,4.541886,3.978088,310.006
4,2018,2460.303520,820.971756,295.578104,35.018974,1308.734686,0.02574,0.347,420.444606,96.600049,4.550562,4.052119,324.343
5,2019,2537.537057,799.265383,328.930336,35.349483,1373.991854,0.02574,0.347,441.409149,97.986091,4.593510,4.117569,343.899
6,2020,2538.745819,635.671702,362.442904,35.856932,1504.774282,0.02574,0.347,483.424362,103.553749,4.659451,4.180064,380.350
7,2021,2675.571993,716.449543,393.691242,36.599743,1528.831464,0.02574,0.347,491.152982,109.003947,4.755976,4.243011,382.662
8,2022,2813.611221,785.544298,422.511919,37.604625,1567.950378,0.02574,0.347,503.720339,114.806528,4.886556,4.310368,389.490
9,2023,2937.819848,843.980733,448.477548,38.973402,1606.388166,0.02574,0.347,516.068877,121.619274,5.064423,4.386026,395.128


In [109]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.347,1312.193118,34.306208,884.575577,231.441998,2462.516902
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.347,1353.612624,34.855388,905.241147,243.981131,2537.690289
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.347,1230.342644,34.816996,830.188301,248.001094,2343.349034
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.347,1274.287681,34.952204,866.909916,268.261581,2444.411382
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.347,1308.734686,35.018974,820.971756,295.578104,2460.303520
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.347,1373.991854,35.349483,799.265383,328.930336,2537.537057
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.347,1504.774282,35.856932,635.671702,362.442904,2538.745819
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.347,1528.831464,36.599743,716.449543,393.691242,2675.571993
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.347,1567.950378,37.604625,785.544298,422.511919,2813.611221
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.347,1606.388166,38.973402,843.980733,448.477548,2937.819848


In [110]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print "Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3))
print "Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3))
print "Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3))

Total corporate tax revenue change, 2014-2027: 0.0
Total corporate tax revenue change, 2017-2026: 0.0
Total corporate tax revenue change, 2018-2027: 0.0


In [111]:
for i in compare:
    print i

5.68434188608e-14
0.0
0.0
5.68434188608e-14
-5.68434188608e-14
-1.13686837722e-13
-1.70530256582e-13
-5.68434188608e-14
-5.68434188608e-14
0.0
-5.68434188608e-14
0.0
-1.13686837722e-13
-2.27373675443e-13


## Calculate percent changes in investment income and pass-through income

In [19]:
execfile('multipliers_for_taxcalc.py')

In [20]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1125.229461,2014,1368.437226,1368.437226,1.0,1.0,1.0,1.0,1.0,1.0
1,1112.097673,2015,1362.951085,1362.951085,1.0,1.0,1.0,1.0,1.0,1.0
2,931.573707,2016,1178.776620,1178.776620,1.0,1.0,1.0,1.0,1.0,1.0
3,965.876547,2017,1214.592554,1214.592554,1.0,1.0,1.0,1.0,1.0,1.0
4,1006.271668,2018,1153.898702,1153.898702,1.0,1.0,1.0,1.0,1.0,1.0
5,1042.720311,2019,1143.947120,1143.947120,1.0,1.0,1.0,1.0,1.0,1.0
6,1078.678771,2020,1043.730562,1043.730562,1.0,1.0,1.0,1.0,1.0,1.0
7,1119.308980,2021,1115.302734,1115.302734,1.0,1.0,1.0,1.0,1.0,1.0
8,1163.545539,2022,1185.894127,1185.894127,1.0,1.0,1.0,1.0,1.0,1.0
9,1209.747833,2023,1243.810693,1243.810693,1.0,1.0,1.0,1.0,1.0,1.0


## Pass changes to Tax-Calculator

In [22]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [23]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

0.0
6.63697694599e-14
6.46208859543e-14
-8.90400431545e-14
5.2461623269e-14
6.66282325216e-14
0.0
7.77076601943e-14
-2.50869831809e-14
8.42405506971e-14
1.01928397244e-14
0.0
8.96756668808e-14
1.73996650506e-13
4.40776197458e-13


In [24]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,0.000000e+00,2014,-1.136868e-13,-1.136868e-13
1,6.636977e-14,2015,0.000000e+00,6.636977e-14
2,6.462089e-14,2016,-2.273737e-13,-1.627528e-13
3,-8.904004e-14,2017,5.684342e-14,-3.219662e-14
4,5.246162e-14,2018,-5.684342e-14,-4.381796e-15
5,6.662823e-14,2019,-1.136868e-13,-4.705861e-14
6,0.000000e+00,2020,-1.705303e-13,-1.705303e-13
7,7.770766e-14,2021,-5.684342e-14,2.086424e-14
8,-2.508698e-14,2022,-5.684342e-14,-8.193040e-14
9,8.424055e-14,2023,-1.136868e-13,-2.944629e-14


In [25]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

-3.21966242937e-14
-4.38179559177e-15
-4.70586052e-14
-1.70530256582e-13
2.08642413335e-14
-8.19304020417e-14
-2.94462870245e-14
-4.66505791364e-14
0.0
-2.40111708409e-14
-5.3377024937e-14


-4.15341479378e-13
